# Learn to rank on top of Azure Cognitive Search

This notebook showcases how to train an L2 ranker, using a [Learn to rank](https://en.wikipedia.org/wiki/Learning_to_rank) approach, to be run on top of Azure Cognitive Search. 

Through this experiment, we are going to:
1. Use Azure Cognitive Search's new feature computation capability to extract text-based similarity features that describe query-to-document relationships
2. Do additional feature engineering to enhance our dataset further 
3. Train a model using [XGBOOST](https://xgboost.readthedocs.io/en/latest/)
4. Evaluate the ranking produced by the trained model against the base Azure Cognitive Search ranking using the [NDCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG).

### Experiment setup

This experiment uses a dataset containing **7102 articles** from the **docs.microsoft.com** website. Each article contains a title, body, description, list of api names and a url path. Articles were augmented using the [key phrase extraction cognitive skill](https://docs.microsoft.com/en-us/azure/search/cognitive-search-skill-keyphrases) as well as with popular search terms that led to those articles. Additionally, we augmented the articles with easy to compute metadata that will be leveraged when training, such as the number of sections and tables in each article, as well as the normalized page views count.

You can find the full index definition at `azs_helpers/index_schema/docs-multilingual-20200217.json`

```json
{
  "name": "docs-multilingual-20200217",
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "facetable": true,
        ...
    },
    ...
  ]
}
```


The experiment also relies on a labeled training set containing over 900 unique queries evaluated against various articles. We refer to this data as the "judgment" list, which will be used as the ground truth when evaluating ranking. Each query is evaluated against 1 to 10 different documents, and for each, provides a "grade" representing how relevant that specific document is to that query. A value of 10 indicates high relevance, while a value of 1 indicates lower relevance.

The judgment list can be found at `data/raw/msft_docs_labels.csv`

#### Configuration
Details and secrets about your search service should be added to a `config.json` file of this format:

```json
{
   "service_name": "YOUR_SERVICE_NAME",
   "endpoint": "https://YOUR_SERVICE_NAME.search.windows.net",
   "api_version": "2019-05-06-preview",
   "api_key": "YOUR_API_KEY",
   "index_name": "msft-docs"
}
```   

The file should be located at `config/config.json`  within the repository. If you prefer, you can uncomment (`Ctrl + /`) & fill in the cell below, and it'll generate a service config for you.

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings(action='once')

from pathlib import Path
import pandas as pd
import azs_helpers.l2r_helper as azs
from azs_helpers.azure_search_client import azure_search_client as azs_client 

service_metadata_config_path = Path.cwd() / 'config' / 'config.json'
azs_service = azs_client.from_json(service_metadata_config_path)

judgement_file_path = Path.cwd() / 'data' / 'raw' / 'msft_docs_labels.csv'
judgements = pd.read_csv(judgement_file_path).drop(['title'], axis=1)

# index creations and ingestion
azs_service.create_index()
azs_service.ingest_documents_from_blob_storage(expected_document_count=7102)

### Extracting features from search service

The following functions are designed to efficiently use the Azure Search service to extract document-query features.

1. We filter each queries to only consider the documents we want to judge. This is achieved by adding a **"filter"** clause to our search query which will restrict the results to the group of documents contained in the group of documents we have judgment values for.
2. We set **"featuresMode"** to "enabled". This will tell the search service to return additional features with the results, including per-field similarity scores.
3. We use the **"select"** clause to only return the url of each documents, as well as a few non-text based fields that could potentially be used as features. This will greatly reduce the amount of data that needs to be transfered between the server and the client.
4. We use the **"searchFields"** parameter to select which text-based fields we want to include in the search process. Those fields will be the only ones for which the service will extract text-based features from (such as per-field similarity).
5. We set the **scoringStatistics** parameter to global, which will give us more consistent and predictable scoring by first making sure all index statistics are are 

The expect response to this query will have the following format:

```json
    "value": [
     {
        "@search.score": 5.1958685,
        "@search.features": {
            "description_en_us": {
                "uniqueTokenMatches": 1.0,
                "similarityScore": 0.29541412
            },
            "body_en_us": {
                "uniqueTokenMatches": 3.0,
                "similarityScore": 0.36644348400000004
            },
            "keyPhrases_en_us": {
                "uniqueTokenMatches": 3.0,
                "similarityScore": 0.35014877
            },
            "title_en_us": {
                "uniqueTokenMatches": 3.0,
                "similarityScore": 1.75451557
            },
            "urlPath": {
                "uniqueTokenMatches": 2.0,
                "similarityScore": 1.07175103
            },
            "searchTerms": {
                "uniqueTokenMatches": 3.0,
                "similarityScore": 1.3575956200000001
            }
        },
        "normalized_pageview": null,
        "tableCount": 0,
        "sectionCount": 7,
        "url_en_us": "https://docs.microsoft.com/en-us/azure/search/"
    }]
```

In [ ]:
def get_search_results_from_service(service, query, urls_filter):
    search_request_body = {
        "search": azs.escape_query(query),
        "featuresMode": "enabled",
        "select": "title_en_us, url_en_us, sectionCount, tableCount, normalized_pageview", 
        "searchFields": "body_en_us,description_en_us,title_en_us,apiNames,urlPath,searchTerms, keyPhrases_en_us",
        "scoringStatistics": "global",
        "top" : 20
    }
    if len(urls_filter) > 0:
        search_request_body["filter"] = " or ".join(f"url_en_us eq '{url}'" for url in urls_filter)

    return service.search(search_request_body)

def get_features_from_service(service, query, group):
    urls = group['url'].values.tolist()
    
    search_results = get_search_results_from_service(service, query, urls)

    # this will flatten the search json response into a panda dataframe
    azs_features = pd.json_normalize(search_results)
    
    # we add the data extracted from azure search to our labeled data by merging them on the "url" field
    merged_results = group.join(azs_features.set_index('url_en_us'), on='url')
    
    # some of the labeled documents in our dataset did not match any documents in the Azure Search instance,
    # we will remove them from our data by dropping any row that did not produce a search score
    return merged_results.dropna(subset=['@search.score'])

### Make parallel calls to the Azure Search service to extract features

To extract all the features from our dataset, we start by grouping the judgment list by query. This will provide us with a list of judged documents for each query. Each call to the Azure Cognitive Search service will use the query from the group,  with filters to make sure we only return the documents from the group. In this dataset, we can expect aproximately 900 queries.

To quickly execute those queries, we setup a thread pool executor which will run the queries in parallel. The level of parallelism can be changed to accomodate different search service capacity.

In [ ]:
%%time

import concurrent
import datetime
from itertools import chain

query_groups = judgements.groupby('query')

executor = concurrent.futures.ThreadPoolExecutor(30)
futures = [executor.submit(get_features_from_service, azs_service, query, group) for (query, group) in query_groups]
concurrent.futures.wait(futures)

all_features = pd.concat([future.result() for future in futures if future], sort=False).fillna(0)

## Visualize our Dataset

Now that the data has finished downloading, let's load our data. Pay attention to a few things in particular:

- **grade** is the ground truth relevance score for this document, where a grade of '10' is most relevant, while a grade of '1' is least relevant. It is an [ordinal variable](https://en.wikipedia.org/wiki/Ordinal_data) ranging from **1 ~ 10**, from least relevant to most relevant.
- **title_en_us** is the English-language title of an article on Microsoft Docs.
- **query** is the search query that led to this document
- All features that begin with an "@" character are returned from the Features API, and will be used extensively by the trained model.
- Some queries have less than 10 documents. We'll have to keep this fact in mind when preparing our data.

In [ ]:
query_groups = all_features.groupby('query')
docs_per_query = query_groups.size()
print(f"Number of unique queries: {len(all_features.groupby('query'))}")
print(f"Number of documents: {len(all_features)}")
print(f"Number of features: {len(all_features.columns)}")
print(f"Fewest docs in a single query: {docs_per_query.min()}")
print(f"Most docs in a single query: {docs_per_query.max()}")

all_features

### Feature engineering

In this section, we will apply additional domain knowledge to add a few features to our model. For example, we are going to add the title length, in words and in characters.

We are also going to drop columns of our dataset that we won't be using for the training process, such as raw url, title and query text. 

In [ ]:
from pprint import pprint

def customize_features(features):
    # adding a query_id column will help us group the data per query later in training
    if 'query_id' not in features:
        features['query_id'] = features.groupby('query').ngroup()
    
    if ('title_en_us' in features):
        features['title_en_us'] = features['title_en_us'].map(lambda title: str(title))
        features['title_length_in_words'] = features['title_en_us'].map(lambda title: float(len(title.split())))
        features['title_length_in_chars'] = features['title_en_us'].map(lambda title: float(len(title)))

    # Discard the columns that we no longer need
    features = features.drop(['query', 'url', 'title_en_us'], axis=1, errors='ignore')
    return features

all_features = customize_features(all_features)
print("All features collected:")
pprint(all_features.columns.values)

all_features.head(5)

### Visualize Azure Cognitive Search NDCG

[**Normalized Discounted Cumulative Gain (NDCG)**](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Discounted_cumulative_gain) is a common metric used to evaluate search ranking systems.  Using `@search.score` returned from the service, we can calculate and visualize Azure Search's NDCG. This will be the baseline to beat with our ranking model.

- `NDCG@K` denotes the NDCG for lists of K length. For example:
    - `NDCG@1` denotes calculating NDCG for a list containing only the top result in a search query
    - `NDCG@5` would refer to the NDCG for a list containing the top 5 results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

query_groups_with_azs_ranking = all_features.sort_values(['@search.score'], ascending=False).groupby('query_id')
ndcg_inputs = [[doc['grade'] for _, doc in group.iterrows()] for query_id, group in query_groups_with_azs_ranking]
original_ndcg = azs.evaluate_ndcg(k_start=1, k_end=10, baseline=ndcg_inputs, plot=True)

## Feature Normalization

Similarity scores are computed to be compared within the context of a single search query, where all documents are evaluated against the same search terms. The range of those scores fluctuates depending on what terms are being searched (for example, common terms often yield to lower similarity scores, since their document frenquency is higher).

To address this problem, we are going to normalize most of our features using the [minmax_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html) scaler, which will be applied at the query level group, so that all documents associated with a particular query will have their feature values scaled from 0 to 1.

For features with outlier values, such as `normalized_pageview`, we will use the [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html).


In [ ]:
from sklearn.preprocessing import minmax_scale, QuantileTransformer
import numpy as np

def normalize_features(features):
    query_groups = features.groupby('query_id')

    for feature in features.columns:
        if feature not in['normalized_pageview', 'grade', 'query_id'] and features.dtypes[feature] in [float, int]:
            features[feature] = query_groups[feature].transform(lambda x: minmax_scale(x.astype(float)))
            
    if 'normalized_pageview' in features.columns:
        features['normalized_pageview'] = query_groups['normalized_pageview'].transform(
            lambda x: np.concatenate(
                QuantileTransformer(n_quantiles=min(len(x), 4)).fit_transform(x.values.reshape(-1,1))
            )
        )
    
    return features
    
df = normalize_features(all_features)
df.head(5)

### Serialize Data for our Experiment

Let's serialize our data to disk. The tutorial will continue in [Part 2: Experiment](./l2r_part2_experiment.ipynb)



In [ ]:
interim_data_dir = Path.cwd() / 'data' / 'interim'
interim_data_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(interim_data_dir / 'normalized_features.csv')